In [1]:
!pip install transformers datasets soundfile speechbrain accelerate gtts

In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset

ds = load_dataset("mozilla-foundation/common_voice_7_0", "ur", use_auth_token=True, split='train')
ds

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 281
})

In [4]:
ds["sentence"]

['پر اپنی محنت سے حاصل کیا ہے',
 'جبکہ مصباح الحق اور یونس خان جیسے سینئرز صرف اس لیے اب تک کھیل رہے ہیں',
 'پنجاب میں صنعتوں اور سی این جی سیکٹرز کو گیس کی فراہمی معطل',
 'لندن فلیٹس کی ملکیت ثابت شدہ ہے۔',
 'کئی کئی برسوں کا فرق ہے۔',
 'تو پاکستان کی سیاسی جماعتیں یہ کیوں نہ کر سکیں؟',
 'جب جنرل راحیل شریف آرمی چیف تھے۔',
 'مگر حیرت کی بات کہ قرآن شریف کو سب سے اوپر رکھا گیا ، گویا سب سےمقدس صحیفہ یہی ہے',
 '"گاؤں جہاں کا قصد تھا ، کا نام ""کلاں"" ہے ۔"',
 'اب بھی وہی قائم ہیں۔',
 'ضیاء الحق دور میں قوم بٹی ہوئی تھی۔',
 'اداکار شاہد کپور کے بارے میں بھی',
 'یہ کسی قوم کی ایک اجتماعی ضرورت کا فطری جواب ہے۔',
 'مجھے دفتر پیدل جانا پڑتا ہے',
 'خدانہ کرے کہ ایسا ہو۔',
 'اپنی چالیسویں سالگرہ کی طرف بڑھتے ہوئے ک',
 'بچن نے اپنی بھابھی ایشوریا کے بارے میں',
 'اب جو اعتراضات اٹھائے گئے ہیں۔',
 'میں فلم کو سمجھ نہیں پایا تھا',
 'اوراب وہ رد ہوچکی۔',
 'اچھی بھی نہیں لگتی۔',
 'لیکن ان کی حکومت شخصی حکومت نہ تھی۔',
 'جس کو صائم الدھر کہا جاتا ہے',
 'پرندے نقل مکانی کرنا شروع کرتے ہیں',
 'اقدار م

In [5]:
from gtts import gTTS
import os
from IPython.display import Audio

def convert_text_to_speech(text, output_file):
    tts = gTTS(text, lang='ur')
    tts.save(output_file)

# Example usage
urdu_text = ds["sentence"][5]
output_file = "output.mp3"

convert_text_to_speech(urdu_text, output_file)
Audio(output_file, rate=16000)
  # Open the output file using the default system audio player


In [6]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")


In [7]:
from IPython.display import Audio
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
import torch
from datasets import load_dataset

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")


speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)


In [13]:
inputs = processor(text="NOAZSHRYF AB AYK NYE TAARF KE SATH SAMNE AARHE HYN۔", return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

Audio(speech, rate=16000)

In [32]:
#convert to roman urdu

buck2uni = {
    "\u0627": "A",
    "\u0627": "A",
    "\u0675": "A",
    "\u0673": "A",
    "\u0630": "A",
    "\u0622": "AA",
    "\u0628": "B",
    "\u067E": "P",
    "\u062A": "T",
    "\u0637": "T",
    "\u0679": "T",
    "\u062C": "J",
    "\u0633": "S",
    "\u062B": "S",
    "\u0635": "S",
    "\u0686": "CH",
    "\u062D": "H",
    "\u0647": "H",
    "\u0629": "H",
    "\u06DF": "H",
    "\u062E": "KH",
    "\u062F": "D",
    "\u0688": "D",
    "\u0630": "Z",
    "\u0632": "Z",
    "\u0636": "Z",
    "\u0638": "Z",
    "\u068E": "Z",
    "\u0631": "R",
    "\u0691": "R",
    "\u0634": "SH",
    "\u063A": "GH",
    "\u0641": "F",
    "\u06A9": "K",
    "\u0642": "K",
    "\u06AF": "G",
    "\u0644": "L",
    "\u0645": "M",
    "\u0646": "N",
    "\u06BA": "N",
    "\u0648": "O",
    "\u0649": "Y",
    "\u0626": "Y",
    "\u06CC": "Y",
    "\u06D2": "E",
    "\u06C1": "H",
    "\u064A": "E",
    "\u06C2": "AH",
    "\u06BE": "H",
    "\u0639": "A",
    "\u0643": "K",
    "\u0621": "A",
    "\u0624": "O",
    "\u060C": ""  # separator ulta comma
}

def transString(string, reverse=0):
    """Given a Unicode string, transliterate into Buckwalter. To go from
    Buckwalter back to Unicode, set reverse=1"""

    for k, v in buck2uni.items():
        if not reverse:
            string = string.replace(k, v)
        else:
            string = string.replace(v, k)

    return string

roman_urdu = []

for text in ds['sentence']:
  word = text
  roman_urdu.append(transString(word))
  # entry['roman_sentence'] = roman_urdu
  # print(roman_urdu)

roman_urdu


['PR APNY MHNT SE HASL KYA HE',
 'JBKH MSBAH ALHK AOR YONS KHAN JYSE SYNYRZ SRF AS LYE AB TK KHYL RHE HYN',
 'PNJAB MYN SNATON AOR SY AYN JY SYKTRZ KO GYS KY FRAHMY MATL',
 'LNDN FLYTS KY MLKYT SABT SHDH HE۔',
 'KYY KYY BRSON KA FRK HE۔',
 'TO PAKSTAN KY SYASY JMAATYN YH KYON NH KR SKYN؟',
 'JB JNRL RAHYL SHRYF AARMY CHYF THE۔',
 'MGR HYRT KY BAT KH KRAAN SHRYF KO SB SE AOPR RKHA GYA  GOYA SB SEMKDS SHYFH YHY HE',
 '"GAON JHAN KA KSD THA  KA NAM ""KLAN"" HE ۔"',
 'AB BHY OHY KAYM HYN۔',
 'ZYAA ALHK DOR MYN KOM BTY HOYY THY۔',
 'ADAKAR SHAHD KPOR KE BARE MYN BHY',
 'YH KSY KOM KY AYK AJTMAAY ZRORT KA FTRY JOAB HE۔',
 'MJHE DFTR PYDL JANA PRTA HE',
 'KHDANH KRE KH AYSA HO۔',
 'APNY CHALYSOYN SALGRH KY TRF BRHTE HOYE K',
 'BCHN NE APNY BHABHY AYSHORYA KE BARE MYN',
 'AB JO AATRAZAT ATHAYE GYE HYN۔',
 'MYN FLM KO SMJH NHYN PAYA THA',
 'AORAB OH RD HOCHKY۔',
 'ACHHY BHY NHYN LGTY۔',
 'LYKN AN KY HKOMT SHKHSY HKOMT NH THY۔',
 'JS KO SAYM ALDHR KHA JATA HE',
 'PRNDE NKL MKANY KRNA SHROA KRTE 

In [35]:
#test the roman text into the model

inputs = processor(text=roman_urdu[5], return_tensors="pt")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

Audio(speech, rate=16000)
